# Kaggle Bird Classifier

### Evaluating model performance

In [19]:
# Mounting Google Drive to access data
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [1]:
# Packages
# Machine Learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator,  DirectoryIterator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation, Dropout, Flatten, Dense
from tensorflow.keras.metrics import TopKCategoricalAccuracy, CategoricalAccuracy,SparseCategoricalCrossentropy
import sklearn.metrics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import CategoricalCrossentropy 

# General 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import time 
import datetime
import io
import itertools
from packaging import version
import sys
import random
import seaborn as sns
%matplotlib inline

# Setting random Seeds
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# Copying zipped data to virtual machine and unzipping, before removing zipped file
zip_path ='/content/drive/My\ Drive/Machine_Learning/Repos/Kaggle_Birds_Classifier/Data/consolidated_zip.zip' 
!cp {zip_path} .
!unzip -q consolidated_zip.zip
!rm consolidated_zip.zip 

In [ ]:
# Load Data
all_path = '/content/consolidated_for_Zip'
class_list = sorted(os.listdir('/content/consolidated_for_Zip')) # Path for Google Colab

# Define generator for validation data
datagen = ImageDataGenerator(
    rescale=1./255,validation_split=0.2,horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1./255,validation_split=0.2)

# Generate batches of images - 
train_gen = DirectoryIterator(
    all_path,datagen,target_size=(224, 224),batch_size=32,subset='training',classes=class_list,seed=42)

valid_gen = DirectoryIterator(
    all_path,val_datagen,target_size=(224, 224),batch_size=32,
    classes=class_list,subset='validation',shuffle=False) 

In [8]:
# Creating Class for Model Evaluation
class Evaluation:
    
    '''
    DESCRIPTION
        This class providesims repeatable methods of evaluating a models performance. 
    
    ARGUMENTS
        model: Keras model
        data_gen: Data generator providing data for model to be evaluated with. 
    
    METHODS: 
        get_preds: calculates both ordinal and one hot encoded predictions
        get_labels: calculates both ordinal and one hot encoded labels
        run_check: Ensures get_preds and get_labels are not rerun
        plot_and_predict: Plots provided image with title of prediction and real label
        print_k_preds: Prints k number of predictions and real labels from user specified index
        print_metrics: Prints statistics on models performance
        get_conf_mat: Calcualtes normalised confusion matrix, true positives (diagonal) 
            and confusion matrix errors (diagonals set to 0)
        plot_conf_mat: Plots normalised confusion matrix and matrix errors (diagonal =0)
        plot_img_ax: Plot image axis (static method)
        img_from_idx: Static method for plotting images from an index
        print_true_pos_k: Sorts true positive rate for each class and prints rate for 
            classes in user defined position
        plot_true_pos_hist: Plots histogram of true positive rates
        plot_numims_Vs_aAcc: Plots categorical accuracy against number of images in the class
        confused_classes_K: Prints classes that are most commonly predicted as a certain other class
        
    '''
    
    # Class Variables 
    #all_path = '/content/consolidated_for_Zip' # Path for Google Colab
    #class_list = sorted(os.listdir('/content/consolidated_for_Zip')) # Path for Google Colab
    all_path = 'data/consolidated'
    class_list = sorted(os.listdir( 'data/consolidated'))

    
    def __init__(self, model,data_gen):
        '''
        Init Method
        
        ARGUMENTS 
            model: Keras Model
            data_gen: Data generator for validation data
            
        OUTPUTS
            pred and labels check set to false
        '''
        self.model = model
        self.data_gen = data_gen
        self.pred_check = False
        self.labels_check = False
  
    def get_preds(self):
        
        '''
        Calculates predictions both ordinal and one hot encoding
            
        '''
        
        self.preds_ohc = self.model.predict_generator(self.data_gen) # One hot encoding predictions
        self.preds = np.argmax(self.preds_ohc, axis=1) # Ordinal encoded predictions
        self.pred_check = True 
        
    def get_labels(self):
        
        '''
        Calculates labels both ordinal and one hot encoding
            
        '''
        
        self.labels = self.data_gen.classes # ordinal labels
        self.labels_ohc = to_categorical(self.labels) # ohc real labels
        self.labels_check = True
    
    def run_check(self):
        
        '''
        Method to check if predictions and labels have been run already.
        To be called in other methods. 
        '''
        
        if self.pred_check == False:
            self.get_preds()
        
        if self.labels_check == False:
            self.get_labels()
        
    
    def plot_and_predict(self,img_array,img_label):
        
        '''
        Plots a given image and gives it a title with the prediction and the real label.
        
        ARGUMENTS
            img_array: Single array for image with shape (224,224,3)
            img_label: Ordinal encoded label
        '''
        
        # Making Title for image
        img_array_for_pred = np.expand_dims(img_array, axis=0) # Expand Dims for predict 
        pred = self.model.predict(img_array_for_pred)
        title = 'Pred : ' + class_list[pred.argmax()] + ' ' + str(round(pred.max()*100,3)) +'%' + ' || Real : ' + class_list[img_label.argmax()]
        
        # Plotting image with title
        image = array_to_img(img_array)
        plt.figure(figsize=(6,6))
        plt.imshow(image)
        plt.axis('off')
        plt.title(title)

    def print_k_preds(self,rint,K):
        
        '''
        Prints a number (k) of instances from given index in predictions 
            where the max is the length of data_gen (rint). 
            
        ARGUMENTS
            rint: Index for instance to be chosen from data generator. Min is K, max is length of generator. 
            K: Number of instances to print
        '''
        
        self.run_check() # Check if get_preds and get_labels have already been run. 
     
        for j in range(rint-5,rint):
            top5_pred = sorted( [(x,i) for (i,x) in enumerate(self.preds_ohc[j])], reverse=True)[:K]
            print('Pred_labels:',(top5_pred),'Real_Label:' , self.labels[j], ' Index:',j)
    
    def print_metrics(self):
        
        '''
        Print metrics: Categorical Crossentropy loss, categorical acuracy and topK categorical accuracy
            for k = 1,3,5
        '''
        
        self.run_check() # Check if get_preds and get_labels have already been run.
        
        # Initiate metrics
        cat_cross_loss = CategoricalCrossentropy()
        cat_acc = CategoricalAccuracy()
        topk_1 = TopKCategoricalAccuracy(k=1)
        topk_3 = TopKCategoricalAccuracy(k=3)
        topk_5 = TopKCategoricalAccuracy(k=5)

        # Print categorical crossentropy
        print('categorical crosstentropy:', 
              cat_cross_loss(self.labels_ohc,self.preds_ohc).numpy())

        # Print categorical accuracy
        cat_acc.update_state(self.labels_ohc,self.preds_ohc)
        print('categorical accuracy:',cat_acc.result().numpy())

        # Print combination of TopK metrics
        topk_1.update_state(self.labels_ohc,self.preds_ohc)
        print('Top 1:',topk_1.result().numpy())
        topk_3.update_state(self.labels_ohc,self.preds_ohc)
        print('Top 3:',topk_3.result().numpy())
        topk_5.update_state(self.labels_ohc,self.preds_ohc)
        print('Top 5:',topk_5.result().numpy())
    
            
    def get_conf_mat(self):
        
        '''
        Calculates a normalised confusion matrix (cm_norm), true positives pulled from the diagonal
        values (true_pos) and the matrix where all diagonals are set to 0 (cm_err). 
        '''
        
        self.run_check() # Check if get_preds and get_labels have already been run.
    
        # Calculate confusion matrix
        cm = sklearn.metrics.confusion_matrix(self.preds, self.labels)

        # Normalise Values in matrix
        rows_sums = cm.sum(axis=1,keepdims=True) # finding total num of images in each class
        self.cm_norm = cm/rows_sums  

        # Calculate diagonals i.e. The True Positive Rate
        self.true_pos = np.diagonal(self.cm_norm)

        # Fill diagonals in with zeros to get an error matrix
        self.cm_err = self.cm_norm.copy() # Making a copy for errors
        np.fill_diagonal(self.cm_err,0) # replace diagonals with 0's. 

    
    def plot_conf_mat(self,title=None):
        
        '''
        Plot confusion matrix with and without the diagonals. 
        '''
        
        self.run_check() # Check if get_preds and get_labels have already been run.
        
        # Calculate confusion matrix
        cm = sklearn.metrics.confusion_matrix(self.preds, self.labels)

        # Normalise Values in matrix
        rows_sums = cm.sum(axis=1,keepdims=True) # finding total num of images in each class
        cm_norm = cm/rows_sums  

        # Calculate diagonals i.e. The True Positive Rate
        tp = np.diagonal(cm_norm)

        # Fill diagonals in with zeros to get a 
        cm_err = cm_norm.copy() # Making a copy for errors
        np.fill_diagonal(cm_err,0) # replace diagonals with 0's.
        
        # Plot confusion matrices 
        figure,[ax1,ax2] = plt.subplots(1,2,figsize=(12, 12))
        im1 = ax1.imshow(cm_norm, interpolation='nearest', cmap=plt.cm.gray)
        im2 = ax2.imshow(cm_err, interpolation='nearest', cmap=plt.cm.gray,vmin=0, vmax=0.2)
        ax1.set_title('Confusion Matrix')
        ax2.set_title('Confusion Matrix Errors')
        figure.suptitle(title)
        figure.tight_layout(rect=[0, 0.03, 1, 1.4]) # Bring suptitle closer to axes
        
    @staticmethod
    def plot_img_ax(img,ax,title=None):
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(title)
     
    @staticmethod
    def img_from_idx(all_path,idx):
        # Function to load image from a given class index
        clss = class_list[idx] # Find class name
        file_list = os.listdir(all_path+'/'+clss) # creates list of files in directory
        file_list_idx = random.randint(0,len(file_list)-1) # Generate random index
        file_path = all_path+'/'+clss+'/'+file_list[file_list_idx] # Join path to image
        img = load_img(file_path) # Load image and return it along with class name
        return img,clss
    
    def print_true_pos_k(self,min_idx,max_idx,step):
        
        '''
        Sorts true positives from low to high and prints the true positive rates between the min
        and max index (min_idx, max_idx). e.g. print_true_pos_k(0,6,1) will 
        print the 6 lowest true positives. 
        '''
    
        self.get_conf_mat() # Calculate True positives
        
        # return indexes of values sorted from min-max
        tp_idxs = self.true_pos.argsort()

        # Loop through range of true positives
        for i in range(min_idx,max_idx,step):
            index = tp_idxs[i]
            print(class_list[index],':',self.true_pos[index])
            img1,title1 = Evaluation.img_from_idx(all_path,index)
            fig,ax1=plt.subplots()
            Evaluation.plot_img_ax(img1,ax1,title1)
    
    def plot_true_pos_hist(self,title=None):
        
        '''
        Plot histogram of true positives
        '''
        
        self.get_conf_mat()
        
        sns.distplot(self.true_pos,bins=20)
        plt.xlim(0, 1)
        plt.xlabel('True Positive Rate')
        plt.title(title)
        plt.show()
        
    def plot_numims_Vs_aAcc(self,title=None):
        
        '''
        Plots scatter of number of instances for each class vs categorical accuracy per class. 
        '''
        
        self.get_conf_mat()
        
        # Initiliase lists
        clss_idxs=[]
        clss_num=[]

        # Loops through the list of classes
        for idx,directory in enumerate(class_list): 
                file_list=os.listdir(all_path + '/' + directory)
                clss_idxs.append(idx)
                clss_num.append(len(file_list)) # Making a list of number of images per class

        # Plot number of images in a class against True Positive Rate
        plt.scatter(clss_num,self.true_pos)
        plt.xlabel('Num of images in class')
        plt.ylabel('True Positive Rate')
        plt.title(title)
        plt.show()
        
    def confused_classes_K(self,K=5):
        
        '''
        prints classes that are most commonly mis predicted an another class
        '''
        
        self.get_conf_mat()
    
        indices = self.cm_err.ravel().argsort() # Get ravelled indices of the values sorted from min-max
        
        for i in range(-1,-K,-1): # loop between -1 and K
    
            id1,id2  = np.unravel_index(indices[i],self.cm_err.shape) # assign index for ith highest value in cm_err
            img1,title1 = Evaluation.img_from_idx(all_path,id1) # Load random images that belong to the relevent class  
            img2,title2 = Evaluation.img_from_idx(all_path,id2)  
            fig,[ax1,ax2] = plt.subplots(1,2) # plot images
            Evaluation.plot_img_ax(img1,ax1,title1)
            Evaluation.plot_img_ax(img2,ax2,title2)
            print('The', title1, 'is predicted as a', title2, # print confusion matrix value as a percent
              round((self.cm_err[id1,id2]*100),2),'% of the time')

In [10]:
help(Evaluation)

Help on class Evaluation in module __main__:

class Evaluation(builtins.object)
 |  Evaluation(model, data_gen)
 |  
 |  DESCRIPTION
 |      This class providesims repeatable methods of evaluating a models performance. 
 |  
 |  ARGUMENTS
 |      model: Keras model
 |      data_gen: Data generator providing data for model to be evaluated with. 
 |  
 |  METHODS: 
 |      get_preds: calculates both ordinal and one hot encoded predictions
 |      get_labels: calculates both ordinal and one hot encoded labels
 |      run_check: Ensures get_preds and get_labels are not rerun
 |      plot_and_predict: Plots provided image with title of prediction and real label
 |      print_k_preds: Prints k number of predictions and real labels from user specified index
 |      print_metrics: Prints statistics on models performance
 |      get_conf_mat: Calcualtes normalised confusion matrix, true positives (diagonal) 
 |          and confusion matrix errors (diagonals set to 0)
 |      plot_conf_mat: Plo

In [ ]:
# Loading saved models
def Path_to_Model(path):
    model = keras.models.load_model(path)
    return model

# Loading Models
%cd /content/drive/My\ Drive/Machine_Learning/Repos/Kaggle_Birds_Classifier/Models/
model_003 = Path_to_Model('CNN--003.200619202528.16--1.571.h5')
model_005 = Path_to_Model('CNN--005.200623090057.08--7.169.h5')
model_006 = Path_to_Model('CNN--006.200627131552.12--0.480--0.897.h5')
model_010 = Path_to_Model('CNN--010.200629140058.07--0.523--0.938.h5')

# Creating Class Objects
model_003 = Evaluation(model_003,valid_gen)   
model_005 = Evaluation(model_005,valid_gen)  
model_006 = Evaluation(model_006,valid_gen)  
model_010 = Evaluation(model_010,valid_gen)  

In [ ]:
# generate example array and label 
img,label = next(valid_gen)
image_array = img[0]
img_label = label[0]

# Plot and predict example image
model_003.plot_and_predict(image_array,img_label)
model_005.plot_and_predict(image_array,img_label)
model_006.plot_and_predict(image_array,img_label)
model_010.plot_and_predict(image_array,img_label)

In [ ]:
print(' ---------------- Model 003 -----------------')
model_003.print_k_preds(888,3)
print(' ---------------- Model 005 -----------------')
model_005.print_k_preds(888,3)
print(' ---------------- Model 006 -----------------')
model_006.print_k_preds(888,3)
print(' ---------------- Model 010 -----------------')
model_010.print_k_preds(888,3)

In [ ]:
print(' ---------------- Model 003 -----------------')
model_003.print_metrics()
print(' ---------------- Model 005 -----------------')
model_005.print_metrics()
print(' ---------------- Model 006 -----------------')
model_006.print_metrics()
print(' ---------------- Model 010 -----------------')
model_010.print_metrics()


In [ ]:
print(' ---------------- Model 003 -----------------')
model_003.plot_conf_mat()
print(' ---------------- Model 005 -----------------')
model_005.plot_conf_mat()
print(' ---------------- Model 006 -----------------')
model_006.plot_conf_mat()
print(' ---------------- Model 010 -----------------')
model_010.plot_conf_mat()


In [ ]:
print(' ---------------- Model 003 -----------------')
model_003.print_true_pos_k(0,6,1)
print(' ---------------- Model 005 -----------------')
model_005.print_true_pos_k(0,6,1)
print(' ---------------- Model 006 -----------------')
model_006.print_true_pos_k(0,6,1)
print(' ---------------- Model 010 -----------------')
model_010.print_true_pos_k(0,6,1)


In [ ]:
plt.xlim(0,1)

In [ ]:
model_003.plot_true_pos_hist(title='Model 003')
model_005.plot_true_pos_hist(title='Model 005')
model_006.plot_true_pos_hist(title='Model 006')
model_010.plot_true_pos_hist(title='Model 010')

In [ ]:
model_003.plot_numims_Vs_aAcc()
model_005.plot_numims_Vs_aAcc()
model_006.plot_numims_Vs_aAcc()
model_010.plot_numims_Vs_aAcc()

In [ ]:
print(' ---------------- Model 003 -----------------')
model_003.confused_classes_K(K=5)
print(' ---------------- Model 005 -----------------')
model_005.confused_classes_K(K=5)
print(' ---------------- Model 006 -----------------')
model_006.confused_classes_K(K=5)
print(' ---------------- Model 010 -----------------')
model_010.confused_classes_K(K=5)